In [40]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling1D,Dense,Embedding
from tensorflow.keras.preprocessing.text import Tokenizer

In [41]:
# corpus=["The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19."]

corpus = ["The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. The reproductive number – the number of secondary infections generated from one infected individual – is understood to be between 2 and 2.5 for COVID-19 virus, higher than for influenza. However, estimates for both COVID-19 and influenza viruses are very context and time-specific, making direct comparisons more difficult."]

In [42]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)
word_index=tokenizer.word_index

In [43]:
sequences=tokenizer.texts_to_sequences(corpus)
X,y=[],[]

In [44]:
for seq in sequences:
    for i,target_word in enumerate(seq):
        for j in range(max(0,i-2),min(i+3,len(seq))):
            if i!=j:
                X.append([target_word,seq[j]])
                y.append(1)

X=np.array(X)
y=np.array(y)

In [45]:
model=Sequential([
    Embedding(input_dim=len(word_index)+1,output_dim=10,input_length=2),
    GlobalAveragePooling1D(),
    Dense(1,activation="sigmoid")
])

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(X,y,epochs=100,verbose=0)

In [49]:
target="the"

if target_word not in word_index:
    print(f"{target} not found")
else:
    target_embedding=model.layers[0].get_weights()[0][word_index[target_word]]

    similarity=[(word,np.dot(target_embedding,model.layers[0].get_weights()[0][index])/(np.linalg(target_embedding) * np.linalg(model.layers[0].get_weights()[0][index]))) for word,index in word_index.items() if word!=target_word]

    related_words=sorted(similarity,key=lambda x: x[1],reverse=True)[:5]

    print(f"Word related to {target_word} => {[word for word,_ in related_words]}")

the not found
